# Assignment 1

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission.*

*Note that this assignment was designed to run in the **Jupyter Notebook** environment. The assignment may not run effectively on Jupyter Lab.*

## NOTE
To successfully complete this homework, it is highly recommended that you install [Anaconda](https://docs.anaconda.com/anaconda/install/) and create a virtual environment specifically for this assignment. Please ensure that you are using Python version 3.10 or higher.

You can create the virtual environment by running the following command:
```bash
conda create --name cs690r python=3.10 numpy scipy pandas matplotlib jupyter
```

Once installed, activate the environment before working on the homework:
```bash
conda activate cs690r
```

The assignment has three tasks:

- Task 1: Understanding the limitations of double-integrating acceleration time-series to obtain position time-series and exploring the efficacy of low-pass filter (LPF) in reducing noice.
- Task 2: Experimenting with different algorithms to obtain gravity-free acceleartion while the sensor is in motion and assessing the associated challenges to gain insights into the complexity of the task.
- Task 3: Experimenting with sensor fusion algorithms to obtain acceleration in the global coordinate system while the sensor is moving randomly.

In [ ]:
# Run some setup code for this notebook
import os
import copy
import numpy as np
import pandas as pd
from types import SimpleNamespace
from scipy.signal import find_peaks, resample, butter, filtfilt
from scipy.integrate import cumulative_trapezoid
from scipy.spatial.transform import Rotation as R

import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook

# The commands will allow the notebook to reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2


[`Attitude and Heading Reference Systems (AHRS)`](https://ahrs.readthedocs.io/en/latest/index.html) is a Python library for sensor fusion, specifically for estimating orientation from inertial measurement unit (IMU) data, which typically includes acceleration and angular velocity measurements.

To download and install the package, you can use the following command in the terminal or command prompt:

```bash
pip install ahrs
```

Refer to this [page](https://ahrs.readthedocs.io/en/latest/installation.html) for more detailed information about ahrs installation. 

Once you install the package, you can import and use the ahrs.filters module for sensor fusion algorithms in you Python scripts or Jupyter notebooks.

In [ ]:
from ahrs.filters import *

[Rockpool Xylo™](https://rockpool.ai/devices/xylo-overview.html) is a family of ultra-low-power (sub-mW) sensory processing and classification devices. In this homework, we are only going to leverage [Xylo™IMU](https://rockpool.ai/devices/xylo-imu/imu-if.html) for IMU data preprocessing.

To download and install the package in the environment you created, you can use the following command in the terminal or command prompt:

```bash
pip install rockpool
```

You will also need `samna` and `bitstruct` packages. 

```bash
pip install samna
pip install bitstruct
```

Refer to this [page](https://rockpool.ai/basics/installation.html) for more detailed information about ahrs installation.

In [ ]:
from cs690r.data_utils import load_sensor_from_csv, load_mocap_from_tsv, trim_data, ARC
from cs690r.plot_utils import plot_time_series, animate_trajectory, compare_trajectory

In [ ]:
# Define gravity constant
GRAVITY_CONSTANT = 9.80665

# Task 1
You can find the data for Task 1 in the "data" folder. In this task, a sensor equipped with accelerometer, gyroscope, and magnetometer was placed on a stationary horizontal surface. To facilitate this process, a function for loading sensor data from text files has been implemented for students. This function is located in the "cs690r.data_utils" module.

You can access the attributes of sensor data using the following code
```python
sample_rate = sensor_data.sample_rate # sampling rate of the sensor, 100Hz
acc = sensor_data.acc # acceleration, unit: m/s^2
gyr = sensor_data.gyr # angular velocity, unit: rad/s
mag = sensor_data.mag # magnetometer data, unit: a.u. (arbitrary units; normalized to earth field strength)

free_acc = sensor_data.free_acc # the sensor's manufacturer (Movella XSens) has its proprietary algorithm to estimate the gravity-free acceleration in the global coordinates
```

Please note that the data in the file is raw and unfiltered.

In [ ]:
# Load stationary data
task1_sensor_file = os.path.join('data', 'task1_sensor_data.csv')
task1_sensor_data = load_sensor_from_csv(task1_sensor_file)

In [ ]:
# Remove gravity from the z-axis, given that the sensor was placed stationary
task1_sensor_data.acc[:, 2] -= GRAVITY_CONSTANT

## **Task 1.1** 

1) Implement the `filtering_and_integrate` function provided below. In this function, use a 6th order Butterworth filter with a cut-off frequency of 8 Hz to low-pass filter the raw acceleration and angular velocity, respectively. Integrate the filtered acceleration to derive the velocity time-series, followed by band-pass filtering (2nd order Butterworth) with cut-off frequency between 0.1 Hz and 8 Hz to address integration drift and high-frequency noise. Repeat the integration and band-pass filtering process to the derived velocity time-series to get position time-series.

2) Apply `filtering_and_integrate` to `task1_sensor_data`. Specifically, double-integrate the acceleration time-series to obtain position time-series both **with and without filtering**.


**Hint**:

1) For integration, consider [`scipy.integrate.cumulative_trapezoid`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.cumulative_trapezoid.html) for integration. You are also free to use other integration implementations you are familiar with.

2) For filtering, checkout `Filtering Example.ipynb` in this folder to learn how to design different types of filters.

In [ ]:
# Exercise 1.1.1: Implement the `filtering_and_integrate` function provided below
import copy
import numpy as np
from scipy.signal import butter, filtfilt
from scipy.integrate import cumulative_trapezoid
def filtering_and_integrate(data, use_filter=True):
    '''
    Input Parameters
    ----------
    data : sensor data object
        To access the acceleration, use:
        acc = data.acc

    use_filter : boolean
        If use_filter is set to True,
        low-pass and band-pass filters will be applied to the data
        
    Output Parameters
    -----------
    data : sensor data object with filtered data
        data.acc: filtered acceleration
        data.vel: integrated, filtered velocity
        data.pos: integrated, filtered position
        data.gyr: filtered angular velocity
    '''
    data = copy.deepcopy(data)

    # Get acceleration
    acc = data.acc.copy()
    
    # Get angular velocity
    gyr = data.gyr.copy() if 'gyr' in dir(data) else None
    
    # Get the sampling rate of the sensor
    sample_rate = data.sample_rate

    ####################################################################
    # TODO: Define low-pass filter and band-pass filter
    # apply the filters on the sensor acceleration and angular velocity
    # if use_filter is True
    # integrate the acceleration to get velocity and position
    def butterworth_filter(x, fs, order, btype, cutoff):
        nyq = 0.5 * fs
        if btype in ["low", "high"]:
            Wn = cutoff / nyq
        elif btype == "band":
            Wn = [cutoff[0] / nyq, cutoff[1] / nyq]
        else:
            raise ValueError(f"Unsupported btype: {btype}")

        b, a = butter(order, Wn, btype=btype, analog=False)
        return filtfilt(b, a, x, axis=0)

    lpf_order = 6
    lpf_cutoff = 8.0  # Hz

    bpf_order = 2
    bpf_cutoff = (0.1, 8.0)  # Hz

    time = data.time if "time" in dir(data) else None

    # 1) Low-pass filter raw acc and gyr
    if use_filter:
        acc = butterworth_filter(acc, sample_rate, lpf_order, "low", lpf_cutoff)
        if gyr is not None:
            gyr = butterworth_filter(gyr, sample_rate, lpf_order, "low", lpf_cutoff)

    # 2) Integrate acc -> vel
    if time is not None:
        vel = cumulative_trapezoid(acc, x=time, axis=0, initial=0.0)
    else:
        vel = cumulative_trapezoid(acc, dx=1.0 / sample_rate, axis=0, initial=0.0)

    # 3) Band-pass filter vel (drift + high-frequency noise control)
    if use_filter:
        vel = butterworth_filter(vel, sample_rate, bpf_order, "band", bpf_cutoff)

    # 4) Integrate vel -> pos
    if time is not None:
        pos = cumulative_trapezoid(vel, x=time, axis=0, initial=0.0)
    else:
        pos = cumulative_trapezoid(vel, dx=1.0 / sample_rate, axis=0, initial=0.0)

    # 5) Band-pass filter pos
    if use_filter:
        pos = butterworth_filter(pos, sample_rate, bpf_order, "band", bpf_cutoff)
    
    ####################################################################
    
    # Save acceleration, velocity, position, and angular velocity
    data.acc = acc
    data.vel = vel
    data.pos = pos
    data.gyr = gyr
    return data

In [ ]:
# Exercise 1.1.2: Apply `filtering_and_integrate` on the task1_sensor_data accelerometer data
# to obtain position time-series. Repeat the process with and without filters.
%matplotlib inline
import matplotlib as mpl
mpl.rcParams["figure.dpi"] = 150

####################################################################
# TODO: Obtain position time-series with filters.
# With filters
task1_filtered = filtering_and_integrate(task1_sensor_data, use_filter=True)
plot_time_series(task1_filtered.pos, "pos", title="Task 1 Position (with filtering)")

####################################################################

####################################################################
# TODO: Obtain position time-series without filters.

# Without filters
task1_nofilter = filtering_and_integrate(task1_sensor_data, use_filter=False)
plot_time_series(task1_nofilter.pos, "pos", title="Task 1 Position (no filtering)")

####################################################################

### Task 1.1 Observations

Without filtering, the double-integrated position drifts rapidly and grows unbounded over time.
This occurs because small accelerometer noise and bias are accumulated twice through numerical integration, producing quadratic drift.

With filtering, the 6th-order low-pass filter removes high-frequency noise from acceleration,
and the 0.1–8 Hz band-pass filtering after each integration suppresses both low-frequency drift and high-frequency noise.
As a result, the estimated position remains smoother and significantly more stable.

However, drift is still present, demonstrating that IMU-only position estimation is inherently unstable over long durations.

## **Task 1.2**

1) Apply `plot_time_series` (located in plot_utils.py) to generate 1D time series plots for acceleration, velocity, and position in each scenario (with and without filtering).

2) Animate the position time-series derived from with/without a low-pass filter using the provided `animate_trajectory` (located in plot_utils.py).

Even though the sensor was placed on a horizontal surface, the raw unfiltered acceleration does not seem to be perfectly zero due to imperfect calibration and inherent sensor noise. Integrating the raw unfiltered acceleration to obtain velocity and position may cause significant drift over time. However, utilizing a low-pass filter and a band-pass filter can effectively mitigate this drift.

In [ ]:
# Exercise 1.2.1: Plot 1D time series for acceleration, velocity, and position
# in each scenario (i.e. with/without filtering) 

####################################################################
# Without filtering
task1_nofilter = filtering_and_integrate(task1_sensor_data, use_filter=False)

plot_time_series(task1_nofilter.acc, "acc", title="Task 1 Acceleration (no filtering)")
plot_time_series(task1_nofilter.vel, "vel", title="Task 1 Velocity (no filtering)")
plot_time_series(task1_nofilter.pos, "pos", title="Task 1 Position (no filtering)")


####################################################################

####################################################################
# With filtering

task1_filtered = filtering_and_integrate(task1_sensor_data, use_filter=True)

plot_time_series(task1_filtered.acc, "acc", title="Task 1 Acceleration (with filtering)")
plot_time_series(task1_filtered.vel, "vel", title="Task 1 Velocity (with filtering)")
plot_time_series(task1_filtered.pos, "pos", title="Task 1 Position (with filtering)")


####################################################################

### Task 1.2.1 Observations

In the no-filter case, acceleration contains small bias and noise.
After the first integration, velocity shows a clear linear drift.
After the second integration, position exhibits strong quadratic drift, growing unbounded over time.

With filtering applied, the acceleration signal is smoother.
The resulting velocity shows significantly reduced drift, and the position remains bounded with much smaller deviation.
This demonstrates how low-pass filtering reduces high-frequency noise,
while band-pass filtering mitigates low-frequency drift introduced during integration.

In [ ]:
# Exercise 1.2.2: Animate the position time-series 
# obtained without filtering and with filtering
# Use "animate_trajectory" from cs690r.data_utils

# Set the view limitsfor X, Y, and Z axis for the 
# animation by passing the correct parameters to 
# animate_trajectory (Refer to the source code 
# for details on parameter usage).  

import matplotlib
matplotlib.use("Agg")

from IPython.display import Image, display

task1_nofilter = filtering_and_integrate(task1_sensor_data, use_filter=False)
pos_nf = task1_nofilter.pos

xmin, xmax = pos_nf[:,0].min(), pos_nf[:,0].max()
ymin, ymax = pos_nf[:,1].min(), pos_nf[:,1].max()
zmin, zmax = pos_nf[:,2].min(), pos_nf[:,2].max()

anim_nf = animate_trajectory(
    pos_nf,
    xmin=xmin, xmax=xmax,
    ymin=ymin, ymax=ymax,
    zmin=zmin, zmax=zmax,
    title="Task 1: No Filter"
)

anim_nf.save("task1_no_filter.gif", writer="pillow", fps=30)
display(Image("task1_no_filter.gif"))


In [ ]:
# For the case where filtering is applied,
# create two animations:
# 1. One animation without specific constraints 
#    on view limits for each axis (i.e., do not 
#    pass parameters to animate_trajectory).
# 2. The other animation that uses the same view 
#    limits as the animation without filtering, 
#    allowing direct comparison.

# With filtering
task1_filtered = filtering_and_integrate(task1_sensor_data, use_filter=True)
pos_f = task1_filtered.pos

anim_f_free = animate_trajectory(
    pos_f,
    title="Task 1: Filtered (Free View)"
)

anim_f_free.save("task1_filter_free.gif", writer="pillow", fps=30)
display(Image("task1_filter_free.gif"))

anim_f_fixed = animate_trajectory(
    pos_f,
    xmin=xmin, xmax=xmax,
    ymin=ymin, ymax=ymax,
    zmin=zmin, zmax=zmax,
    title="Task 1: Filtered (Fixed View)"
)

anim_f_fixed.save("task1_filter_fixed.gif", writer="pillow", fps=30)
display(Image("task1_filter_fixed.gif"))


### Task 1.2.2 Observations

In the no-filter case, the trajectory rapidly drifts away from the origin due to accumulated integration errors.
The motion appears unbounded, especially in the X and Y directions.

In the filtered case without fixed view limits, the trajectory appears stable and tightly bounded near the origin.

When using the same view limits as the no-filter case, the filtered trajectory is visibly much smaller in magnitude,
highlighting how filtering significantly reduces integration drift.

This demonstrates that low-pass filtering removes high-frequency noise,
while band-pass filtering mitigates low-frequency bias and drift introduced during double integration.

# Task 2
In this task, an 9-axis inertial measurement unit (referred to as *sensor*) was moved along a rectangular trajectory within 3D space (i.e., the trajectory discussed in class). The sensor captured the three-axis accelerometer, three-axis gyroscope, and three-axis magnetometer time-series. Moreover, a reflective marker was also placed on the sensor to capture the three-axis position time-series using a motion caption system (referred to as *mocap*). 

Note that we have used *hand clapping* to synchronize the mocap and sensor data.

You can find the data for Task 2 in the "data" folder. The sensor data were saved in `task_2_sensor_data.csv`, whereas the mocap data were saved in `task2_mocap_data.tsv`. You may use `load_sensor_from_csv` and `load_mocap_from_tsv` functions provided in `data_utils.py` to load the data.

You can access the attributes of mocap data by using the following code
```python
sample_rate = mocap_data.sample_rate # sampling rate of the mocap system, 150Hz
raw_pos = mocap_data.raw_pos # position, unit: m
```

Please note that the data in the files are raw and unfiltered.

In [ ]:
# Load sensor data
task2_sensor_file = os.path.join('data', 'task2_sensor_data.csv')
task2_sensor_data = load_sensor_from_csv(task2_sensor_file)

# Load mocap data
task2_mocap_file = os.path.join('data', 'task2_mocap_data.tsv')
task2_mocap_data = load_mocap_from_tsv(task2_mocap_file)

Run the following code to visualize the raw acceleration and angular velocity captured by the sensor. Note that the data collector clapped multiple times before and after the data collection, resulting in large peaks in the data.

In [ ]:
print("Sensor sample rate:", task2_sensor_data.sample_rate)
print("Sensor acc shape:", task2_sensor_data.acc.shape)
print("Sensor gyr shape:", task2_sensor_data.gyr.shape)
print("task2_mocap_data_keys", task2_mocap_data.__dict__.keys())

print("\nMocap sample rate:", task2_mocap_data.sample_rate)
print("Mocap raw position shape:", task2_mocap_data.pos.shape)

# Plot the sensor's raw acceleration and angular velocity
n_rows = 2
n_cols = 1
row_sz = 3
col_sz = 7

# %matplotlib notebook
%matplotlib inline
fig = plt.figure(figsize=(n_cols*col_sz, n_rows*row_sz))
ax = fig.add_subplot(211)
ax.plot(task2_sensor_data.acc)
ax.set_xlabel('Frame')
ax.set_ylabel('Acc ($m/s^2$)')
ax.set_title('Sensor raw acceleration')
ax.legend(['X-axis', 'Y-axis', 'Z-axis'])

ax = fig.add_subplot(212)
ax.plot(task2_sensor_data.gyr)
ax.set_xlabel('Frame')
ax.set_ylabel('Angular vel ($rad/s$)')
ax.set_title('Sensor raw angular velocity')
ax.legend(['X-axis', 'Y-axis', 'Z-axis'])
fig.tight_layout()
plt.show()

## **Task 2.1**

1) Implement the `filtering_and_gradient` function provided below. In this function, use a 6th order Butterworth filter with a cut-off frequency of 8 Hz to low-pass filter the raw position captured by the mocap system. Compute the gradient of the filtered position to derive the velocity time-series, followed by the same low-pass filter. Repeat the process of taking gradient and low-pass filtering to the velocity times-series to acquire acceleration time-series.

2) Apply `filtering_and_gradient` to `task2_mocap_data`. 

3) Apply `filtering_and_integrate` from Task 1 to `task2_sensor_data`.

**Hint**

1) For gradient, you can use [`numpy.gradient`](https://numpy.org/doc/stable/reference/generated/numpy.gradient.html). You are also free to use other integration implementations you are familiar with.

In [ ]:
# Exercise 2.1.1 Implement the `filtering_and_gradient` function provided below
def filtering_and_gradient(data):
    '''
    Input Parameters
    ----------
    data : mocap data object
        To access the position, use:
        pos = data.pos
        
    Output Parameters
    -----------
    data : mocap data object with filtered data
        data.pos: filtered position
        data.vel: filtered velocity
        data.acc: filtered position
    '''
    data = copy.deepcopy(data)
    
    # Get raw position
    pos = data.pos.copy()
    
    # Get the sampling rate of the sensor
    sample_rate = data.sample_rate
    
    ####################################################################
    # TODO: Define low-pass filter
    # apply the filters on the data
    # take the gradient of the position to get velocity and acceleration

    from scipy import signal
    import numpy as np

    # 6th order Butterworth low-pass @ 8 Hz
    cutoff_hz = 8.0
    b_lp, a_lp = signal.butter(
        N=6,
        Wn=cutoff_hz,
        btype="low",
        fs=sample_rate
    )

    # 1) low-pass filter raw position
    pos = signal.filtfilt(b_lp, a_lp, pos, axis=0)

    # 2) gradient to get velocity, then low-pass
    dt = 1.0 / sample_rate
    vel = np.gradient(pos, dt, axis=0)
    vel = signal.filtfilt(b_lp, a_lp, vel, axis=0)

    # 3) gradient to get acceleration, then low-pass
    acc = np.gradient(vel, dt, axis=0)
    acc = signal.filtfilt(b_lp, a_lp, acc, axis=0)
    
    ####################################################################
    
    # Save filtered acceleration, velocity, and position
    data.pos = pos
    data.vel = vel
    data.acc = acc
    return data

In [ ]:
# Exercise 2.1.2: Apply `filtering_and_gradient` on task2_mocap_data 
task2_mocap_filtered = filtering_and_gradient(task2_mocap_data)

print("Filtered mocap pos:", task2_mocap_filtered.pos.shape)
print("Filtered mocap vel:", task2_mocap_filtered.vel.shape)
print("Filtered mocap acc:", task2_mocap_filtered.acc.shape)
plot_time_series(task2_mocap_filtered.pos, "pos", title="Task 2 Mocap Position (filtered)")
plot_time_series(task2_mocap_filtered.vel, "vel", title="Task 2 Mocap Velocity (filtered, gradient)")
plot_time_series(task2_mocap_filtered.acc, "acc", title="Task 2 Mocap Acceleration (filtered, 2nd gradient)")

In [ ]:
# Exercise 2.1.3: Apply `filtering_and_integrate` on task2_sensor_data
task2_sensor_filtered = filtering_and_integrate(task2_sensor_data, use_filter=True)

print("Filtered sensor acc:", task2_sensor_filtered.acc.shape)
print("Integrated sensor vel:", task2_sensor_filtered.vel.shape)
print("Integrated sensor pos:", task2_sensor_filtered.pos.shape)

plot_time_series(task2_sensor_filtered.acc, "acc", title="Task 2 Sensor Acceleration (filtered)")
plot_time_series(task2_sensor_filtered.vel, "vel", title="Task 2 Sensor Velocity (integrated + filtered)")
plot_time_series(task2_sensor_filtered.pos, "pos", title="Task 2 Sensor Position (double integrated + filtered)")

## **Task 2.2**


In this task, you will delve into synchronizing data captured the sensor and mocap. Variations in sampling rates or other factors often introduce discrepancies in the collected data. To facilitate synchronization, the data collector performed multiple claps while holding the sensor (which was attached with a mocap marker) before and after data collection. This resulted in large peaks in acceleration, as shown in our previous plots. Leveraging these claps, we can align different data by trimming segments between the claps and resampling to a uniform sampling rate.

1) Run the provided code to trim both sensor and mocap data.

2) Implement the `resampling_mocap` function. In the function, use [`scipy.signal.resample`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.resample.html) to resample the mocap acceleration, velocity, and position, in order to match the sampling rate of the sensor data. We opt to reduce the sampling rate of the mocap data to align with that of the sensor data, aiming to maintain data integrity (compared to upsampling the sensor data to match the mocap data, which entails data interpolation and may compromise integrity).

3) Apply `resampling_mocap` to `task2_mocap_data`.


In [ ]:
# Build derived signals first (creates .vel and .pos)
task2_sensor_proc = filtering_and_integrate(task2_sensor_data, use_filter=True)

# trim_data() also expects free_acc; if you don't have it yet, just set it for now
if not hasattr(task2_sensor_proc, "free_acc"):
    task2_sensor_proc.free_acc = task2_sensor_proc.acc.copy()

# Mocap also should be processed first (creates .vel and .acc)
task2_mocap_proc = filtering_and_gradient(task2_mocap_data)

# Exercise 2.2.1: Trim both sensor and mocap data
sensor_start, sensor_end = 1183, 4431  # task 2
task2_sensor_proc = trim_data(task2_sensor_proc, sensor_start, sensor_end, "sensor")

mocap_start, mocap_end = 1981, 6812  # task 2
task2_mocap_proc = trim_data(task2_mocap_proc, mocap_start, mocap_end, "mocap")
print("Sensor:", task2_sensor_proc.acc.shape, task2_sensor_proc.vel.shape, task2_sensor_proc.pos.shape)
print("Mocap :", task2_mocap_proc.pos.shape, task2_mocap_proc.vel.shape, task2_mocap_proc.acc.shape)

In [ ]:
# Exercise 2.2.2: Implement the `resampling_mocap` function provided below
def resampling_mocap(data, num):
    '''
    Input Parameters
    ----------
    data : mocap data object
    
    num : int, the number of samples in the resampled signal
    
    Output Parameters
    ----------
    data : mocap data object with resampled pos, vel and acc
    '''
    data = copy.deepcopy(data)
    
    ####################################################################
    # TODO: resample acceleration, velocity, and position
    from scipy.signal import resample

    original_len = data.pos.shape[0]

    data.pos = resample(data.pos, num, axis=0)
    data.vel = resample(data.vel, num, axis=0)
    data.acc = resample(data.acc, num, axis=0)

    data.sample_rate = data.sample_rate * (num / original_len)
    ####################################################################
    
    return data

In [ ]:
# Exercise 2.2.3: Apply `resampling_mocap` to `task2_mocap_data`.
# Number of samples should match trimmed sensor length
num_samples = task2_sensor_proc.pos.shape[0]

task2_mocap_resampled = resampling_mocap(task2_mocap_proc, num_samples)

print("Resampled Mocap:",
      task2_mocap_resampled.pos.shape,
      task2_mocap_resampled.vel.shape,
      task2_mocap_resampled.acc.shape)

plt.figure(figsize=(10,4))

sensor_x = task2_sensor_proc.acc[:,0]
mocap_x = task2_mocap_resampled.acc[:,0]

sensor_x = (sensor_x - sensor_x.mean()) / sensor_x.std()
mocap_x = (mocap_x - mocap_x.mean()) / mocap_x.std()

plt.plot(sensor_x, label="Sensor X (normalized)")
plt.plot(mocap_x, label="Mocap X (normalized)")
plt.legend()
plt.title("Acceleration Alignment (Normalized)")
plt.show()


In practical applications, removing gravitational components from local acceleration to obtain gravity-free acceleration is essential. **Task 2.3 – Task 2.7** will explore different approaches to achieve this:
1. Applying a highpass filter to remove gravitational components.
2. Using Autocorrelation-based Rotation Correction (ARC) algorithm to estimate rotation matrix
3. Applying Rodrigues' rotation matrix to the residual component of the highpass filter to estimate local gravity and correct coordinate.
4. Fusing local acceleration with angular velocity to estimate sensor orientation in global coordinate system where Z-axis aligns with gravity.
5. Using the sensor fusion algorithm provided by the XSens.

Finally, in **Task 2.8**, you will compare the performance of each method with the ground truth (i.e., mocap data).

## **Task 2.3**
A high-pass filter removes low-frequency components from local acceleration signals, effectively attenuating gravitational effects. ThisåΩ task will guide you through implementing and applying a band-pass filter, which integrates a high-pass filter to reduce gravitational influence and a low-pass filter to minimize sensor noise, ultimately estimating gravity-free acceleration.

1) Implement the `bpf_remove_gravity` function. First, define a $6^{th}$ order Butterworth band-pass filter with cut-off frequency between $[0.1Hz, 20Hz]^{[1]}$. Second, apply the band-pass filter on the local acceleration to obtain gravity-free acceleration. Finally, apply `filtering_and_integrate` from Task 1 to the transformed gravity-free acceleration to derive velocity and position time-series.
åçç
2) Apply `bpf_remove_gravity` to `task2_sensor_data` to obtain gravity-free acceleration, velocity, and position time-series.  

3) Use `compare_trajectory` to visualize and compare the gravity-free acceleration, velocity, and position from  the sensor with the data motion capture (mocap) systems.  

**Reference**

[1] [Gupta, Anoopum S., et al. "At-home wearables and machine learning sensitively capture disease progression in amyotrophic lateral sclerosis." Nature Communications 14.1 (2023): 5080.](https://www.nature.com/articles/s41467-023-40917-3)

In [ ]:
# Exercise 2.3.1 Implement the `bpf_remove_gravity` function
from types import SimpleNamespace
import copy
import numpy as np
from scipy.signal import butter, sosfiltfilt
def bpf_remove_gravity(local_data):
    '''
    Input Parameters
    ----------
    local_data : sensor data object in sensor's coordinates
    
    Output Parameters
    ----------
    gravity_free_data : compute gravity-free data
        gravity_free_data.sample_rate: the synchornized sampling rate
        gravity_free_data.acc: gravity-free 3D accelerometer
    '''
    
    local_data = copy.deepcopy(local_data)
    
    local_acc = local_data.acc
    local_gyr = local_data.gyr
    sample_rate = local_data.sample_rate
    
    ####################################################################
    # TODO: Define and apply a Butterworth band-pass filter to obtain
    # gravity-free inertial data from local acceleration

    order = 6
    f_low = 0.1
    f_high = 20.0

    nyq = 0.5 * sample_rate
    low = f_low / nyq
    high = f_high / nyq

    # safety in case sample_rate is low (rare, but avoids butter() errors)
    if high >= 1.0:
        high = 0.99
    if low <= 0.0:
        low = 1e-6

    sos = butter(order, [low, high], btype="bandpass", output="sos")
    gravity_free_acc = sosfiltfilt(sos, local_acc, axis=0)
    
    ####################################################################
    
    # Save the transformed data
    gravity_free_data = SimpleNamespace()
    gravity_free_data.sample_rate = local_data.sample_rate
    gravity_free_data.acc = gravity_free_acc
    
    ####################################################################
    # TODO: Apply filtering_and_integrate to gravity_free_data
    gravity_free_data.gyr = local_gyr
    gravity_free_data.free_acc = gravity_free_acc.copy()
    gravity_free_data = filtering_and_integrate(gravity_free_data, use_filter=False)

    ####################################################################

    return gravity_free_data

In [ ]:
# Exercise 2.3.2: Apply `bpf_remove_gravity` on `task2_sensor_data`
task2_bpf = bpf_remove_gravity(task2_sensor_proc)
print("BPF gravity-free acc:", task2_bpf.acc.shape)
print("BPF velocity:", task2_bpf.vel.shape)
print("BPF position:", task2_bpf.pos.shape)

plot_time_series(task2_bpf.acc, "acc", title="Task 2 Sensor Acc (BPF gravity-free)")
plot_time_series(task2_bpf.vel, "vel", title="Task 2 Sensor Vel (BPF integrated)")
plot_time_series(task2_bpf.pos, "pos", title="Task 2 Sensor Pos (BPF double integrated)")

In [ ]:
# Exercise 2.3.3: Compare acceleration, velocity, 
# and position from BPF and mocap system

compare_trajectory(task2_bpf, task2_mocap_resampled, title1="Task 2: BPF gravity-free", title2="Task 2: mocap")


### Comparison Between BPF-Based Estimation and Mocap Ground Truth

To evaluate the effectiveness of the band-pass filtering (BPF) approach, the gravity-free acceleration, velocity, and position obtained from the sensor were compared against the motion capture (mocap) ground truth.

**Acceleration Comparison:**
The BPF-processed acceleration aligns temporally with the mocap acceleration, indicating successful synchronization. The major motion events occur at similar frames. However, the sensor signal remains noisier and exhibits slight amplitude discrepancies due to residual bias and measurement noise.

**Velocity Comparison:**
After integrating the gravity-free acceleration, noticeable drift appears in the sensor-derived velocity. In contrast, the mocap velocity remains bounded and stable. This indicates that small residual errors in acceleration accumulate during integration.

**Position Comparison:**
The divergence becomes more significant at the position level. The sensor-derived position shows substantial drift and unrealistic displacement, while the mocap trajectory remains physically consistent and bounded. This demonstrates how integration amplifies even minor acceleration errors.

**Conclusion:**
Although band-pass filtering reduces gravitational influence and preserves motion timing, it does not completely eliminate bias. The remaining residual errors accumulate during integration, resulting in significant velocity and position drift. Therefore, band-pass filtering alone is insufficient for robust inertial trajectory estimation when compared to mocap ground truth.

## **Task 2.4**
This task explores the Autocorrelation-based Rotation Correction (`ARC`) technique to suppress gravity components using only three-axis acceleroemeter.

1) Run the following code and apply `ARC` to `task2_sensor_data` to obtain gravity-free acceleration, velocity and position time-series. You can find the source code of `ARC` in `cs690r/data_utils.py`.

2) Use `compare_trajectory` to visualize and compare the gravity-free acceleration, velocity, and position from  the sensor with the data motion capture (mocap) systems. 

In [ ]:
# Exercise 2.4.1 Run the `ARC` function
# from cs690r.data_utils import ARC
# print(ARC)
# import sys
# print(sys.version)
# import numpy as np
# import rockpool
# print("numpy:", np.__version__)
# print("rockpool:", rockpool.__version__)
import copy
import numpy as np

sensor_start, sensor_end = 1183, 4431

task2_sensor_raw_trim = copy.deepcopy(task2_sensor_data)
task2_sensor_raw_trim.acc = task2_sensor_raw_trim.acc[sensor_start:sensor_end]
if hasattr(task2_sensor_raw_trim, "gyr") and task2_sensor_raw_trim.gyr is not None:
    task2_sensor_raw_trim.gyr = task2_sensor_raw_trim.gyr[sensor_start:sensor_end]
if hasattr(task2_sensor_raw_trim, "time") and task2_sensor_raw_trim.time is not None:
    task2_sensor_raw_trim.time = task2_sensor_raw_trim.time[sensor_start:sensor_end]

task2_ARC_data = ARC(task2_sensor_raw_trim)
task2_ARC_data = filtering_and_integrate(task2_ARC_data)

print("ARC acc:", task2_ARC_data.acc.shape)
print("ARC vel:", task2_ARC_data.vel.shape)
print("ARC pos:", task2_ARC_data.pos.shape)


In [ ]:
# Exercise 2.4.2: Compare acceleration, velocity, 
# and position from ARC and mocap system

num_samples_arc = task2_ARC_data.pos.shape[0]
task2_mocap_resampled_arc = resampling_mocap(task2_mocap_proc, num_samples_arc)

compare_trajectory(
    task2_ARC_data,
    task2_mocap_resampled_arc,
    title1="Task 2: ARC gravity-free",
    title2="Task 2: mocap"
)


Task 2.4.2 Comparison (ARC vs Mocap)

ARC removes most of the gravity component, and the resulting acceleration falls in a realistic range comparable to mocap. The ARC-derived velocity follows the same overall motion trends as mocap but is smoother and slightly smaller due to filtering and integration effects. The largest mismatch is in position: ARC position shows drift and does not match the clean mocap trajectory because double integration amplifies noise and small biases. Overall, ARC improves acceleration and velocity estimates, while mocap remains the more reliable ground truth for position.

## **Task 2.5**
This task explores a method where local acceleration is estimated using a low-pass filter, and Rodrigues’ rotation matrix is used to transform local acceleration into a global reference frame.

1) Implement `rodrigues_gravity_removal`. First, implement the algorithm. You can find a detailed description of the algorithm in the lecture slides. Second, apply the algorithm to the local acceleration to obtain gravity-free acceleration. Finally, apply `filtering_and_integrate` from Task 1 to the gravity-free acceleration to derive velocity and position time-series.

2) Apply `rodrigues_gravity_removal` to `task2_sensor_data` to obtain gravity-free acceleration, velocity, and position time-series.

3) Utilize `compare_trajectory` to visualize the gravity-free acceleration, velocity, and position from both the sensor and mocap systems.

**Hint**

1. You can use [`np.linalg.norm`](https://numpy.org/doc/2.2/reference/generated/numpy.linalg.norm.html) to calculate the norm of a vector.

2. You can use [`np.cross`](https://numpy.org/doc/stable/reference/generated/numpy.cross.html) to calculate the cross product between two vectors.

3. You can use [`np.arccos`](https://numpy.org/doc/2.2/reference/generated/numpy.arccos.html) to calculate trigonometric inverse cosine, element-wise.

In [ ]:
# Exercise 2.5.1 Implement the `rodrigues_gravity_removal` function
import numpy as np
m = task2_sensor_data.acc[:300].mean(axis=0)
print(m, "norm:", np.linalg.norm(m))

def rodrigues_gravity_removal(local_data):
    '''
    Input Parameters
    ----------
    local_data : sensor data object in sensor's coordinates
    
    Output Parameters
    ----------
    gravity_free_data : compute gravity-free data
        gravity_free_data.sample_rate: the synchornized sampling rate
        gravity_free_data.acc: gravity-free 3D accelerometer
    '''
    
    local_data = copy.deepcopy(local_data)
    
    local_acc = local_data.acc
    local_gyr = local_data.gyr
    sample_rate = local_data.sample_rate
    
    ####################################################################
    # TODO: Implement rodrigues_gravity_removal

    dt = 1.0 / sample_rate
    tau = 2.0
    alpha = tau / (tau + dt)

    # 1. Estimate gravity direction in local frame using Low-Pass Filter
    g_est = np.zeros_like(local_acc)
    g_est[0] = local_acc[0]
    for i in range(1, len(local_acc)):
        g_est[i] = alpha * g_est[i-1] + (1 - alpha) * local_acc[i]

    g_ref = np.array([0.0, 0.0, 1.0])  # Target global Z-axis
    I = np.eye(3)
    eps = 1e-8
    gravity_free_acc = np.zeros_like(local_acc)

    for i in range(len(local_acc)):
        # Normalize local gravity estimate to get direction unit vector
        g_vec = g_est[i]
        g_mag = np.linalg.norm(g_vec)

        if g_mag < eps:
            gravity_free_acc[i] = local_acc[i]
            continue

        a = g_vec / g_mag  # Vector to rotate FROM (local gravity)
        b = g_ref          # Vector to rotate TO (global Z)

        v = np.cross(a, b)
        s = np.linalg.norm(v)
        c = np.dot(a, b)

        # Calculate Rodrigues Rotation Matrix
        if s < eps:
            # If vectors are already aligned or opposite
            R = I if c > 0 else -I
        else:
            # Skew-symmetric matrix
            K = np.array([
                [0,    -v[2],  v[1]],
                [v[2],  0,    -v[0]],
                [-v[1], v[0],  0]
            ])
            # R = I + [v]x + [v]x^2 * (1-c)/s^2
            R = I + K + (K @ K) * ((1 - c) / (s**2))

        # 2. Transform the RAW local acceleration into the Global frame
        acc_global = R @ local_acc[i]

        # 3. Subtract global gravity (9.81 m/s^2) from the Global Z component
        g_mag_actual = np.linalg.norm(g_est[i])
        acc_global[2] -= g_mag_actual

        gravity_free_acc[i] = acc_global
    
    ####################################################################
    
    # Save the transformed data
    gravity_free_data = SimpleNamespace()
    gravity_free_data.sample_rate = local_data.sample_rate
    gravity_free_data.acc = gravity_free_acc
    
    ####################################################################
    # TODO: Apply filtering_and_integrate to gravity_free_data
    gravity_free_data = filtering_and_integrate(gravity_free_data)
    ####################################################################

    return gravity_free_data

In [ ]:
# Exercise 2.5.2: Apply `rodrigues_gravity_removal` on `task2_sensor_data`
task2_rodrigues_data = rodrigues_gravity_removal(task2_sensor_data)

# Print shapes and stats to verify the transformation
print("Shapes (acc, vel, pos):",
      task2_rodrigues_data.acc.shape,
      task2_rodrigues_data.vel.shape,
      task2_rodrigues_data.pos.shape)



In [ ]:
# Exercise 2.5.3: Compare acceleration, velocity, 
num_samples = task2_rodrigues_data.acc.shape[0]
task2_mocap_resampled = resampling_mocap(task2_mocap_proc, num_samples)

start, end = 250, 4600

task2_rodrigues_trim = copy.deepcopy(task2_rodrigues_data)
task2_rodrigues_trim.acc = task2_rodrigues_data.acc[start:end]
task2_rodrigues_trim.vel = task2_rodrigues_data.vel[start:end]
task2_rodrigues_trim.pos = task2_rodrigues_data.pos[start:end]

task2_mocap_trim = copy.deepcopy(task2_mocap_resampled)
task2_mocap_trim.acc = task2_mocap_resampled.acc[start:end]
task2_mocap_trim.vel = task2_mocap_resampled.vel[start:end]
task2_mocap_trim.pos = task2_mocap_resampled.pos[start:end]
compare_trajectory(
    task2_rodrigues_trim,
    task2_mocap_trim,
    title1="Task 2: Rodrigues Trimmed",
    title2="Task 2: Mocap Trimmed"
)


Task 2.5.3 – Comparison of Rodrigues Gravity Removal and Mocap

After trimming the unstable initial and final segments, the Rodrigues-based gravity removal produces acceleration signals with a magnitude comparable to the mocap ground truth. The overall amplitude range is similar, although the sensor-based acceleration appears noisier due to IMU measurement noise and imperfect gravity estimation.

The velocity derived from the sensor follows the general motion trends observed in the mocap data but exhibits more oscillation and slight bias. This behavior is expected since velocity is obtained by integrating noisy acceleration, which amplifies small estimation errors.

The position comparison shows noticeable drift and deviation from the clean step-like structure of the mocap trajectory. This is a fundamental limitation of double integration of IMU data, where even small biases accumulate over time.

Overall, the Rodrigues gravity removal method effectively suppresses gravity and produces physically reasonable acceleration estimates. However, integration drift and sensor noise result in reduced accuracy compared to mocap measurements, particularly in position estimation.

## **Task 2.6**

This task will demonstrate the process of fusing acceleration and angular velocity data collected in the sensor's coordinate system to estimate the sensor's orientation in the global coordinate system. Once the orientation is determined, this information can be utilized to align or rotate the sensor to the global coordinate system.

1) Implement the `fuse_and_rotate` function. Firstly, althrough `ahrs` supports a variety of algorithms for estimating the sensor's orientation, we opted to use the AQUA algorithm. Secondly, utilize the estimated orientation to transform the acceleration and angular velocity from the sensor's coordinate system to the global coordinates. Thirdly, remove gravity from the vertical axis of the global coordinate acceleration. Finally, apply `filtering_and_integrate` from Task 1 to the transformed gravity-free, global-coordinate acceleration to derive velocity and position time-series in the global coordinates.

2) Apply `fuse_and_rotate` to `task2_sensor_data` to obtain acceleration, velocity, and position time-series in the global coordinates.

3) Utilize `compare_trajectory` to visualize the acceleration, velocity, and position from both the sensor and mocap systems in the global coordinates.

**Hint**

1) You can choose from the following methods provided by AHRS to estimate sensor orientation. The methods shown in the table below assume that the Z-axis of the global coordinate system aligns with gravity. Therefore, you can easily eliminate gravity from the transformed acceleration.

| Fusion Method                                                           | Orientatio<br>Representation | What to set inverse in<br>`R.from_quat().apply()` |
|-------------------------------------------------------------------------|----------------------------|------------------------------------------------|
| [AQUA](https://ahrs.readthedocs.io/en/latest/filters/aqua.html)         | global -> local            | True                                           |
| [Madgwich](https://ahrs.readthedocs.io/en/latest/filters/madgwick.html) | local -> global            | False                                          |
| [Mahony](https://ahrs.readthedocs.io/en/latest/filters/mahony.html)     | local -> global            | False                                          |

Here's an example of sensor orientation estimation using `AQUA` from `ahrs`. Suppose you have acceleration data `acc` in m/s^2 and angular velocity data `gyr` in rad/s, sampled at a frequency of `sample_freq` Hz. To estimate the orientation, you can follow these steps:

```python
ahrs_filter = AQUA(acc=acc, gyr=gyr, frequency=sample_freq) # define sensor fusion method
orientation = ahrs_filter.Q # access the orientation
```

The `orientation` calculated by `AQUA` represents the global to local transformation, whereas for Madgwick and Mahony, it represents the local to global transformation. You need to be careful with this difference when using them for future coordinate transformation.

2) For coordinate transformation, you can use [`scipy.spatial.transform.Rotation`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html). Assume the local acceleration captured in the sensor local frame is `local_acc` and the `orientation` transforms the data from the sensor local frame to the global frame. To perform transformation,

```python
orientation = orientation[:, [1, 2, 3, 0]] # the orientation calculated from ahrs has different form than what is required by R.from_quat

global_acc = R.from_quat(orientation).apply(local_acc, inverse=True) # inverse needs to be set to `True` because the orientation calcuated from AQUA represents the transformation from global coordinate system to the sensor coordinate system. For Madgwich and Mahony methods, `inverse` needs to be set to `False`.

global_acc[:, 2] -= GRAVITY_CONSTANT # remove gravity from Z-axis
```

In the code above, if `inverse` is specificed `True` in `R.from_quat().apply()`, the inverse of the rotation(s) is applied to the input vectors. You can refer to the table above for guidance on how to set up this parameter.

In [ ]:
# Exercise 2.6.1 Implement the `fuse_and_rotate` function
def fuse_and_rotate(local_data):
    '''
    Input Parameters
    ----------
    local_data : sensor data object in sensor's coordinates
    
    Output Parameters
    ----------
    global_data : transformed data object in global coordinates
        global_data.sample_rate: the synchornized sampling rate
        global_data.acc: global 3D accelerometer
        global_data.gyr: global 3D gyroscope
    '''
    
    local_data = copy.deepcopy(local_data)
    
    local_acc = local_data.acc
    local_gyr = local_data.gyr
    sample_rate = local_data.sample_rate
    
    ####################################################################
    # TODO: Calculate sensor orientation, then use orientation to 
    # transform acceleration and angular velocity from sensor local 
    # frame to global frame
    # Remove gravity from global acceleration
    from ahrs.filters import AQUA
    from scipy.spatial.transform import Rotation as R

    GRAVITY_CONSTANT = 9.81

    # 1) Sensor fusion (AQUA) to estimate orientation (global -> local)
    ahrs_filter = AQUA(acc=local_acc, gyr=local_gyr, frequency=sample_rate)
    orientation = ahrs_filter.Q  # shape: (N, 4) in [w, x, y, z]

    # 2) Reorder quaternion to scipy format: [x, y, z, w]
    orientation_xyzw = orientation[:, [1, 2, 3, 0]]

    # 3) Rotate local acc and gyr into global frame
    # AQUA gives global->local, so to map local->global we apply inverse=True
    global_acc = R.from_quat(orientation_xyzw).apply(local_acc, inverse=True)
    global_gyr = R.from_quat(orientation_xyzw).apply(local_gyr, inverse=True)

    # 4) Remove gravity from global Z axis
    global_acc[:, 2] -= GRAVITY_CONSTANT

    ####################################################################
    
    # Save the transformed data
    global_data = SimpleNamespace()
    global_data.sample_rate = local_data.sample_rate
    global_data.acc = global_acc
    global_data.gyr = global_gyr
    
    ####################################################################
    # TODO: Apply filtering_and_integrate to global data

    global_data = filtering_and_integrate(global_data)
    ####################################################################
    
    return global_data

In [ ]:
# Exercise 2.6.2: Apply `fuse_and_rotate` on `task2_sensor_data`

task2_fused_global = fuse_and_rotate(task2_sensor_data)
print(task2_fused_global.acc.shape, task2_fused_global.vel.shape, task2_fused_global.pos.shape)


In [ ]:
# Exercise 2.6.3: Compare acceleration, velocity, and position 
# from sensor fusion algorithm and mocap system

# 1) Resample mocap to match fused sensor length
num_samples = task2_fused_global.acc.shape[0]
task2_mocap_resampled = resampling_mocap(task2_mocap_proc, num_samples)

# 2) Trim both (adjust start/end if needed)
start, end = 250, 4600

task2_fused_trim = copy.deepcopy(task2_fused_global)
task2_fused_trim.acc = task2_fused_global.acc[start:end]
task2_fused_trim.vel = task2_fused_global.vel[start:end]
task2_fused_trim.pos = task2_fused_global.pos[start:end]

task2_mocap_trim = copy.deepcopy(task2_mocap_resampled)
task2_mocap_trim.acc = task2_mocap_resampled.acc[start:end]
task2_mocap_trim.vel = task2_mocap_resampled.vel[start:end]
task2_mocap_trim.pos = task2_mocap_resampled.pos[start:end]

# 3) Compare trimmed
compare_trajectory(
    task2_fused_trim,
    task2_mocap_trim,
    title1="Task 2: Sensor Fusion Trimmed (Global)",
    title2="Task 2: Mocap Trimmed (Global)"
)

## Task 2.6 Trimmed Results Analysis

After trimming the unstable beginning and ending segments, the sensor fusion results were compared with mocap ground truth.

### Acceleration
The transformed global acceleration aligns well with mocap data. Motion transitions are clearly captured, and gravity removal on the Z-axis is successful.

### Velocity
Velocity follows the general motion pattern of the mocap system. However, noticeable drift is present during stationary periods due to small accelerometer bias and integration errors.

### Position
Position estimation captures overall motion trends but exhibits significant drift and amplitude mismatch. This is expected because position is obtained through double integration of acceleration, which amplifies small bias errors.

### Conclusion
Sensor fusion improves orientation estimation and global transformation, but pure IMU integration without bias correction leads to velocity and position drift. Additional techniques such as zero-velocity updates or Kalman filtering would be required for accurate long-term position tracking.

## **Task 2.7**

In this exercise, you will utilize gravity-free acceleration data from the manufacturer's (XSens) proprietary sensor fusion algorithm, which is recognized for its accuracy in integrating multi-modal data to estimate sensor orientation and derive acceleration in global coordinates. This will enable you to evaluate the manufacturer's algorithm performance and compare it with mocap data. Note that the sensor fusion algorithm by Xsens uses all nine-axis inertial data (accelerometer + gyroscope + magnetometer) compared to the AQUA algorithm that uses only six-axis data (accelerometer + gyroscope).

1) Apply the `filtering_and_integrate` function to the manufacturer gravity-free acceleration data to derive velocity and position time-series.

2) Utilize `compare_trajectory` to visualize the acceleration, velocity, and position from both the manufacturer's gravity-free data and mocap systems in the global coordinates. 

In [ ]:
task2_xsens_data = SimpleNamespace()
task2_xsens_data.sample_rate = task2_sensor_data.sample_rate
task2_xsens_data.acc = task2_sensor_data.free_acc
task2_xsens_data.gyr = task2_sensor_data.gyr

In [ ]:
# Exercise 2.7.1: Apply the `filtering_and_integrate` function 
# to the xsens gravity-free acceleration data

task2_xsens_proc = filtering_and_integrate(task2_xsens_data)

print("XSens acc:", task2_xsens_proc.acc.shape)
print("XSens vel:", task2_xsens_proc.vel.shape)
print("XSens pos:", task2_xsens_proc.pos.shape)


In [ ]:
# Exercise 2.7.2: Compare acceleration, velocity, and position 
# from xsens and mocap system
num_samples = task2_xsens_proc.acc.shape[0]
task2_mocap_resampled = resampling_mocap(task2_mocap_proc, num_samples)

start, end = 250, 4600

task2_xsens_trim = copy.deepcopy(task2_xsens_proc)
task2_xsens_trim.acc = task2_xsens_proc.acc[start:end]
task2_xsens_trim.vel = task2_xsens_proc.vel[start:end]
task2_xsens_trim.pos = task2_xsens_proc.pos[start:end]

task2_mocap_trim = copy.deepcopy(task2_mocap_resampled)
task2_mocap_trim.acc = task2_mocap_resampled.acc[start:end]
task2_mocap_trim.vel = task2_mocap_resampled.vel[start:end]
task2_mocap_trim.pos = task2_mocap_resampled.pos[start:end]

compare_trajectory(
    task2_xsens_trim,
    task2_mocap_trim,
    title1="Task 2: XSens Trimmed (global)",
    title2="Task 2: Mocap Trimmed (global)"
)

## Task 2.7 Results Analysis (XSens vs Mocap)

After applying filtering and integration to the manufacturer-provided gravity-free acceleration, the XSens results were compared with mocap ground truth.

### Acceleration
XSens global acceleration aligns closely with mocap data. Motion transitions are well captured and noise levels are low. Gravity removal is accurate.

### Velocity
Velocity estimation from XSens shows significantly reduced drift compared to the previous methods (Rodrigues and AQUA). Stationary periods remain close to zero, indicating improved bias handling.

### Position
Position tracking closely follows the mocap trajectory pattern with noticeably reduced drift. Although small integration errors remain, the amplitude and motion timing are much closer to ground truth.

### Conclusion
The XSens 9-axis fusion algorithm provides substantially better global motion estimation than the 6-axis AQUA and Rodrigues approaches. The inclusion of magnetometer data improves heading stability and reduces integration drift, resulting in more accurate velocity and position estimation.

## **Task 2.8**

In this exercise, you will evaluate the performance of the results from **Task 2.3 - 2.7** both quantitatively and qualitatively.

1) Define the function that calculate the root mean square error (RMSE) and the normalized root mean square error (NRMSE) between two given time-series. NRMSE is obtained by normalizing the RMSE to the range of ground truth, so that the error can be represented in percentage (%). 

2) Apply your function and compute RMSE and NRMSE between the gravity-free acceleration, velocity, and position time-series obtained from **Task 2.3 - 2.7** with the mocap data, respectively. The ground truth is mocap data.

This analysis will provide insights into the accuracy and reliability of your sensor fusion algorithm compared to the ground truth mocap data.

3) Create animations using `animate_trajectory` for the three position time-series derived from **Task 2.3 - 2.7** as well as the one captured by the mocap system, respectively. By visually comparing these animations, you can qualitatively assess how well the sensor fusion algorithm aligns with the mocap system in representing the sensor's movements.

In [ ]:
# Exercise 2.8.1: Define the function for calculating
# RMSE and NRMSE between two given time series

import numpy as np

def rmse_nrmse(pred, gt, eps=1e-12):
    """
    pred, gt: arrays shaped (T, 3) for x/y/z
    returns:
      rmse_axis: (3,)
      nrmse_axis_pct: (3,) in %
      rmse_all: scalar (over all axes)
      nrmse_all_pct: scalar in %
    """
    pred = np.asarray(pred)
    gt = np.asarray(gt)
    assert pred.shape == gt.shape, f"Shape mismatch: {pred.shape} vs {gt.shape}"

    err = pred - gt
    rmse_axis = np.sqrt(np.mean(err**2, axis=0))  # per axis

    # normalize by ground-truth range per axis
    gt_range_axis = (gt.max(axis=0) - gt.min(axis=0))
    nrmse_axis_pct = (rmse_axis / (gt_range_axis + eps)) * 100.0

    # overall (all axes together)
    rmse_all = np.sqrt(np.mean(err**2))
    gt_range_all = (gt.max() - gt.min())
    nrmse_all_pct = (rmse_all / (gt_range_all + eps)) * 100.0

    return rmse_axis, nrmse_axis_pct, rmse_all, nrmse_all_pct


def print_metrics(tag, pred_obj, gt_obj):
    for name in ["acc", "vel", "pos"]:
        pred = getattr(pred_obj, name)
        gt = getattr(gt_obj, name)
        rmse_axis, nrmse_axis_pct, rmse_all, nrmse_all_pct = rmse_nrmse(pred, gt)
        print(f"\n{tag} | {name.upper()}")
        print(f"  RMSE axis [x y z]: {rmse_axis}")
        print(f"  NRMSE axis % [x y z]: {nrmse_axis_pct}")
        print(f"  RMSE all: {rmse_all:.6f}")
        print(f"  NRMSE all %: {nrmse_all_pct:.3f}")

import copy

def resample_and_trim(sensor_obj, mocap_proc, start=None, end=None):
    # 1) resample mocap to sensor length
    n = sensor_obj.acc.shape[0]
    mocap_rs = resampling_mocap(mocap_proc, n)

    # 2) optional trimming
    if start is None or end is None:
        return sensor_obj, mocap_rs

    sensor_trim = copy.deepcopy(sensor_obj)
    mocap_trim = copy.deepcopy(mocap_rs)

    for f in ["acc", "vel", "pos"]:
        setattr(sensor_trim, f, getattr(sensor_obj, f)[start:end])
        setattr(mocap_trim, f, getattr(mocap_rs, f)[start:end])

    return sensor_trim, mocap_trim


In [ ]:
# Exercise 2.8.2.1: Use your function to caculate RMSE and NRMSE
# between the acceleration, velocity, and position from Task 2.3
# and mocap data

# Print RMSE and NRMSE values

start, end = 250, 4600  # keep consistent with your earlier comparisons

task23_s, mocap23 = resample_and_trim(task2_bpf, task2_mocap_proc, start, end)
print_metrics("Task 2.3 (BPF)", task23_s, mocap23)




In [ ]:
# Exercise 2.8.2.2: Use your function to caculate RMSE and NRMSE
# between the acceleration, velocity, and position from Task 2.4
# and mocap data

# Print RMSE and NRMSE values

task24_s, mocap24 = resample_and_trim(task2_ARC_data, task2_mocap_proc, start, end)
print_metrics("Task 2.4 (ARC)", task24_s, mocap24)


In [ ]:
# Exercise 2.8.2.3: Use your function to caculate RMSE and NRMSE
# between the acceleration, velocity, and position from Task 2.5
# and mocap data

# Print RMSE and NRMSE values
task25_s, mocap25 = resample_and_trim(task2_rodrigues_data, task2_mocap_proc, start, end)
print_metrics("Task 2.5 (Rodrigues)", task25_s, mocap25)


In [ ]:
# Exercise 2.8.2.4: Use your function to caculate RMSE and NRMSE
# between the acceleration, velocity, and position from Task 2.6
# and mocap data

# Print RMSE and NRMSE values
task26_s, mocap26 = resample_and_trim(task2_fused_global, task2_mocap_proc, start, end)
print_metrics("Task 2.6 (AQUA Fusion Global)", task26_s, mocap26)


In [ ]:
# Exercise 2.8.2.5: Use your function to caculate RMSE and NRMSE
# between the acceleration, velocity, and position from Task 2.7
# and mocap data

# Print RMSE and NRMSE values

task2_xsens_proc = filtering_and_integrate(task2_xsens_data)
task27_s, mocap27 = resample_and_trim(task2_xsens_proc, task2_mocap_proc, start, end)
print_metrics("Task 2.7 (XSens Global)", task27_s, mocap27)


In [ ]:
# Exercise 2.8.3.1: Animate the position time-series
# captured by the motion capture system
import numpy as np
import matplotlib
matplotlib.use("Agg")

from IPython.display import Image, display

start, end = 250, 4600
fps = 30
out_gif = "task2_mocap.gif"

mocap_pos = np.asarray(task2_mocap_proc.pos)[start:end]
ani = animate_trajectory(mocap_pos, title="2.8.3.1: Mocap Position")
ani.save(out_gif, writer="pillow", fps=fps)
display(Image(filename=out_gif))


In [ ]:
# Exercise 2.8.3.2: Animate the position time-series
# derived from Task 2.3
import numpy as np
import matplotlib
matplotlib.use("Agg")

from IPython.display import Image, display

start, end = 250, 4600
fps = 30
out_gif = "task23_bpf.gif"

ani = animate_trajectory(task23_s.pos, title="2.8.3.2: Task 2.3 (BPF)")
ani.save(out_gif, writer="pillow", fps=30)

display(Image(filename=out_gif))


In [ ]:
# Exercise 2.8.3.3: Animate the position time-series 
# derived from Task 2.4

out_gif = "task24_arc.gif"

ani = animate_trajectory(task24_s.pos, title="2.8.3.3: Task 2.4 (ARC)")
ani.save(out_gif, writer="pillow", fps=30)

display(Image(filename=out_gif))



In [ ]:
# Exercise 2.8.3.4: Animate the position time-series 
# derived from Task 2.5

out_gif = "task25_rodrigues.gif"

ani = animate_trajectory(task25_s.pos, title="2.8.3.4: Task 2.5 (Rodrigues)")
ani.save(out_gif, writer="pillow", fps=30)

display(Image(filename=out_gif))


In [ ]:
# Exercise 2.8.3.5: Animate the position time-series 
# derived from Task 2.6
out_gif = "task26_aqua.gif"

ani = animate_trajectory(task26_s.pos, title="2.8.3.5: Task 2.6 (AQUA Fusion)")
ani.save(out_gif, writer="pillow", fps=30)

display(Image(filename=out_gif))


In [ ]:
# Exercise 2.8.3.6: Animate the position time-series 
# derived from Task 2.7
out_gif = "task27_xsens.gif"

ani = animate_trajectory(task27_s.pos, title="2.8.3.6: Task 2.7 (XSens)")
ani.save(out_gif, writer="pillow", fps=30)

display(Image(filename=out_gif))


# Task 3
You can find the data for Task 3 in the "data" folder. In this task, a nine-axis IMU and a mocap marker was placed on a human subject's wrist, who performed some arbitrary, patternless, and random upper-limb movements (e.g., swinging the arm in the air randomly). The sensor data were saved in `task_3_sensor_data.csv`, and the mocap data were saved in `task3_mocap_data.tsv`.

In [ ]:
# Load sensor data
task3_sensor_file = os.path.join('data', 'task3_sensor_data.csv')
task3_sensor_data = load_sensor_from_csv(task3_sensor_file)

# Load mocap data
task3_mocap_file = os.path.join('data', 'task3_mocap_data.tsv')
task3_mocap_data = load_mocap_from_tsv(task3_mocap_file)

The objective of Task 3 is to apply the abovementioned procedures (e.g., **Task 2.1 - 2.8**) to this dataset, aiming to assess the algorithm's capability to handle entirely unpredictable movements that simulate daily scenarios.

## **Task 3.1**
Apply `filtering_and_gradient` to `task3_mocap_data`. 

In [ ]:
# Exercise 3.1: Apply `filtering_and_gradient` to `task3_mocap_data`
task3_mocap_proc = filtering_and_gradient(task3_mocap_data)

print("Task3 mocap acc:", task3_mocap_proc.acc.shape)
print("Task3 mocap vel:", task3_mocap_proc.vel.shape)
print("Task3 mocap pos:", task3_mocap_proc.pos.shape)


## **Task 3.2**
Apply `filtering_and_integrate` to `task3_sensor_data` with filtering. 

In [ ]:
# Exercise 3.2: Apply `filtering_and_integrate` to `task3_sensor_data` with filtering. 
task3_sensor_proc = filtering_and_integrate(task3_sensor_data, use_filter=True)

print("Task3 sensor acc:", task3_sensor_proc.acc.shape)
print("Task3 sensor vel:", task3_sensor_proc.vel.shape)
print("Task3 sensor pos:", task3_sensor_proc.pos.shape)

## **Task 3.3**
Synchronization and resampling. Trim the sensor data and mocap between the provided clapping indices. Then, apply `resampling_mocap` to resample to the mocap data to match the sampling rate of sensor data.

In [ ]:
# Exercise 3.3: Synchronization and resampling
task3_sensor_proc = filtering_and_integrate(task3_sensor_data, use_filter=True)
task3_mocap_proc  = filtering_and_gradient(task3_mocap_data)
sensor_start, sensor_end = 918, 5198
task3_sensor_proc = trim_data(task3_sensor_proc, sensor_start, sensor_end, 'sensor')

mocap_start, mocap_end = 1300, 7719
task3_mocap_proc = trim_data(task3_mocap_proc, mocap_start, mocap_end, 'mocap')

####################################################################
# TODO: Apply `resampling_mocap` to resample to the mocap data
n = task3_sensor_proc.acc.shape[0]
task3_mocap_resampled = resampling_mocap(task3_mocap_proc, n)

print("Sensor pos:", task3_sensor_proc.pos.shape)
print("Resampled mocap pos:", task3_mocap_resampled.pos.shape)
####################################################################


## **Task 3.4**
1) Repeat Task 2.3 and apply `bpf_remove_gravity` on the sensor data.

2) Repeat Task 2.4 and apply `ARC` on the sensor data.

3) Repeat Task 2.5 and apply `rodrigues_gravity_removal` on the sensor data.

4) Repeat Task 2.6 and apply `fuse_and_rotate` on the sensor data.

5) Repeat Task 2.7 and obtain the data provided by the xsens.

In [ ]:
# Exercise 3.4.1
####################################################################
# TODO: Apply `bpf_remove_gravity` on the sensor data.
task3_bpf = bpf_remove_gravity(task3_sensor_data)
#task3_bpf = filtering_and_integrate(task3_bpf)

####################################################################

####################################################################
# TODO: Compare acceleration, velocity, and position 
# from Exercise 3.4.1 and mocap system
n = task3_bpf.acc.shape[0]
task3_mocap_bpf = resampling_mocap(task3_mocap_proc, n)
%matplotlib inline

compare_trajectory(
    task3_bpf,
    task3_mocap_bpf,
    title1="Task 3: BPF gravity-free",
    title2="Task 3: mocap"
)

####################################################################

In [ ]:
# Exercise 3.4.2
####################################################################
# TODO: Run `xyloIMU` on the sensor data.
import copy

# 1) Run xyloIMU if it exists, else use ARC
if "xyloIMU" in globals():
    task3_xylo = xyloIMU(copy.deepcopy(task3_sensor_data))
else:
    task3_xylo = ARC(copy.deepcopy(task3_sensor_data))

# 2) Integrate to get vel/pos
task3_xylo = filtering_and_integrate(task3_xylo)

# 3) Resample mocap to match
n = task3_xylo.acc.shape[0]
task3_mocap_xylo = resampling_mocap(task3_mocap_proc, n)
####################################################################

####################################################################
# TODO: Compare acceleration, velocity, and position 
# from Exercise 3.4.2 and mocap system
compare_trajectory(
    task3_xylo,
    task3_mocap_xylo,
    title1="Task 3: xyloIMU / ARC gravity-free",
    title2="Task 3: mocap"
)

####################################################################

In [ ]:
# Exercise 3.4.3
####################################################################
# TODO: Apply `rodrigues_gravity_removal` on the sensor data.
task3_rodrigues = rodrigues_gravity_removal(task3_sensor_data)

n = task3_rodrigues.acc.shape[0]
task3_mocap_rodrigues = resampling_mocap(task3_mocap_proc, n)
####################################################################

####################################################################
# TODO: Compare acceleration, velocity, and position 
# from Exercise 3.4.3 and mocap system
compare_trajectory(
    task3_rodrigues,
    task3_mocap_rodrigues,
    title1="Task 3: Rodrigues gravity-free",
    title2="Task 3: mocap"
)
####################################################################

In [ ]:
# Exercise 3.4.4
####################################################################
# TODO: Apply `fuse_and_rotate` on the sensor data.
import copy

task3_fused = fuse_and_rotate(copy.deepcopy(task3_sensor_data))

n = task3_fused.acc.shape[0]
task3_mocap_fused = resampling_mocap(task3_mocap_proc, n)
####################################################################

####################################################################
# TODO: Compare acceleration, velocity, and position 
# from Exercise 3.4.4 and mocap system
compare_trajectory(
    task3_fused,
    task3_mocap_fused,
    title1="Task 3: AQUA Fusion (Global)",
    title2="Task 3: mocap"
)
####################################################################

In [ ]:
task3_xsens_data = SimpleNamespace()
task3_xsens_data.sample_rate = task3_sensor_data.sample_rate
task3_xsens_data.acc = task3_sensor_data.free_acc
task3_xsens_data.gyr = task3_sensor_data.gyr

In [ ]:
# Exercise 3.4.5
####################################################################
# TODO: Apply `filter_and_integrate` 
# on task3_xsens_data
task3_xsens_proc = filtering_and_integrate(task3_xsens_data)

n = task3_xsens_proc.acc.shape[0]
task3_mocap_xsens = resampling_mocap(task3_mocap_proc, n)
####################################################################

####################################################################
# TODO: Compare acceleration, velocity, and position 
# from Exercise 3.4.5 and mocap system
compare_trajectory(
    task3_xsens_proc,
    task3_mocap_xsens,
    title1="Task 3: XSens (Global)",
    title2="Task 3: mocap"
)
####################################################################

## **Task 3.5**
Repeat **Task 2.6** on `task3_sensor_data` and 

1) Calculate the RMSE and NRMSE between the acceleration, velocity, and position time-series derived from **Task 3.4** and the mocap system, respectively.

2) Create animations using `animate_trajectory` for the position time-series derived from **Task 3.4** and captured by the mocap system, respectively.

In [ ]:
# Exercise 3.5.1.1 Calculate RMSE and NRMSE between 
# the acceleration, velocity, and position from 
# bpf_remove_gravity and mocap data

# Print RMSE and NRMSE values

import numpy as np
import copy

def rmse_nrmse(pred, gt, eps=1e-12):
    pred = np.asarray(pred)
    gt = np.asarray(gt)
    assert pred.shape == gt.shape, f"Shape mismatch: {pred.shape} vs {gt.shape}"

    err = pred - gt
    rmse_axis = np.sqrt(np.mean(err**2, axis=0))

    gt_range_axis = gt.max(axis=0) - gt.min(axis=0)
    nrmse_axis_pct = (rmse_axis / (gt_range_axis + eps)) * 100.0

    rmse_all = np.sqrt(np.mean(err**2))
    gt_range_all = gt.max() - gt.min()
    nrmse_all_pct = (rmse_all / (gt_range_all + eps)) * 100.0

    return rmse_axis, nrmse_axis_pct, rmse_all, nrmse_all_pct


def print_metrics(tag, pred_obj, gt_obj):
    for name in ["acc", "vel", "pos"]:
        pred = getattr(pred_obj, name)
        gt = getattr(gt_obj, name)

        rmse_axis, nrmse_axis_pct, rmse_all, nrmse_all_pct = rmse_nrmse(pred, gt)

        print(f"\n{tag} | {name.upper()}")
        print(f"  RMSE axis [x y z]: {rmse_axis}")
        print(f"  NRMSE axis % [x y z]: {nrmse_axis_pct}")
        print(f"  RMSE all: {rmse_all:.6f}")
        print(f"  NRMSE all %: {nrmse_all_pct:.3f}")


def resample_mocap_to_method(mocap_proc, method_obj):
    n = method_obj.acc.shape[0]
    return resampling_mocap(mocap_proc, n)

task3_mocap_bpf_m = resample_mocap_to_method(task3_mocap_proc, task3_bpf)
print_metrics("Task 3.4.1 (BPF)", task3_bpf, task3_mocap_bpf_m)


In [ ]:
# Exercise 3.5.1.2 Calculate RMSE and NRMSE between 
# the acceleration, velocity, and position from 
# xyloIMU and mocap data

# Print RMSE and NRMSE values
task3_mocap_xylo_m = resample_mocap_to_method(task3_mocap_proc, task3_xylo)
print_metrics("Task 3.4.2 (xyloIMU/ARC)", task3_xylo, task3_mocap_xylo_m)


In [ ]:
# Exercise 3.5.1.3 Calculate RMSE and NRMSE between 
# the acceleration, velocity, and position from 
# rodrigues_gravity_removal and mocap data

# Print RMSE and NRMSE values
task3_mocap_rod_m = resample_mocap_to_method(task3_mocap_proc, task3_rodrigues)
print_metrics("Task 3.4.3 (Rodrigues)", task3_rodrigues, task3_mocap_rod_m)


In [ ]:
# Exercise 3.5.1.4 Calculate RMSE and NRMSE between 
# the acceleration, velocity, and position from 
# fuse_and_rotate and mocap data


# Print RMSE and NRMSE values
task3_mocap_fused_m = resample_mocap_to_method(task3_mocap_proc, task3_fused)
print_metrics("Task 3.4.4 (AQUA Fusion)", task3_fused, task3_mocap_fused_m)


In [ ]:
# Exercise 3.5.1.5 Calculate RMSE and NRMSE between 
# the acceleration, velocity, and position from 
# xsens-provided gravity free data and mocap data

# Print RMSE and NRMSE values
task3_mocap_xsens_m = resample_mocap_to_method(task3_mocap_proc, task3_xsens_proc)
print_metrics("Task 3.4.5 (XSens)", task3_xsens_proc, task3_mocap_xsens_m)


In [ ]:
# Exercise 3.5.2.1: Animate the position time-series 
# captured by the mocap system
%matplotlib widget
ani_mocap = animate_trajectory(
    task3_mocap_proc.pos,
    title="Task 3: Mocap Position"
)
ani_mocap


In [ ]:
# Exercise 3.5.2.2: Animate the position time-series 
# derived from the bpf_remove_gravity
ani_bpf = animate_trajectory(
    task3_bpf.pos,
    title="Task 3.4.1: BPF Position"
)
ani_bpf


In [ ]:
# Exercise 3.5.2.3: Animate the position time-series 
# derived from the ARC
ani_xylo = animate_trajectory(
    task3_xylo.pos,
    title="Task 3.4.2: xyloIMU / ARC Position"
)
ani_xylo


In [ ]:
# Exercise 3.5.2.4: Animate the position time-series 
# derived from the rodrigues_gravity_removal
ani_rodrigues = animate_trajectory(
    task3_rodrigues.pos,
    title="Task 3.4.3: Rodrigues Position"
)
ani_rodrigues


In [ ]:
# Exercise 3.5.2.5: Animate the position time-series 
# derived from the fuse_and_rotate
ani_aqua = animate_trajectory(
    task3_fused.pos,
    title="Task 3.4.4: AQUA Fusion Position"
)
ani_aqua


In [ ]:
# Exercise 3.5.2.6: Animate the position time-series 
# derived from the xsens data
ani_xsens = animate_trajectory(
    task3_xsens_proc.pos,
    title="Task 3.4.5: XSens Position"
)
ani_xsens
